# Получение данных о графе: узлы и отношения

**_Данный код всё ещё не позволяет полноценно работать с графом в проекте по анализу общественного транспорта. Возникают ошибки при добавлении графа в базу данных._** 

В данном файле мы рассмотрим возможности получения информации об остановках в виде узлов и об отношениях в виде маршрутов автобусов. Предполагаем обработку области по имени города.

Автор рекомендует использовать второй способ получения узлов, поскольку по его мнению записями в таком виде манипулировать проще. Тем не менее, описаны оба варианта

Ниже приведены используемые библиотеки

In [ ]:
!pip install osmnx
!pip install pandas

In [1]:
import osmnx as ox   #работа с OpenStreetMap
import pandas as pd   #работа с датафреймами
import networkx as nx   #работа с графами

### Получение узлов с помощью features_from_place  _(не рекомендуется)_

Для получения информации об узлах используем функцию, возвращающую всю информацию о передаваемой области : **`features_from_place`**_`(query, tags, which_result=None)`_. Результатом её выполнения будет GeoDataFrame с информацией об узлах

Далее воспользуемся самой функцией **`features_from_place`**. В качестве тега будем использовать параметр _`public_transport`_ и значение _`stop_position`_. 

In [ ]:
city = 'Санкт-Петербург'
nodes = ox.features_from_place(city, tags={"public_transport": "stop_position"})
#print(nodes)
for idx, row in nodes.iterrows():
    print(idx)

Также есть возможность получать данные с помощью запроса к **`Overpass-API`**

In [94]:
pd.set_option('display.max_columns', None)    #отображение всех столбцов
print(nodes.columns)

AttributeError: 'dict' object has no attribute 'columns'

Среди атрибутов полученного датафрейма можно отметить:
- _bus_ : является ли остановка автобусной (yes/NaN). Дополнительно хранит osmid остановки
- _name_ : название остановки
- _trolleybus_ : является ли остановка троллейбусной
- _geometry_ : координаты точки. Значение типа point, для обработки лучше перевести в отдельные поля
- _tram_ : является ли остановка трамвайной
- _subway_ : является ли остановка станцией метро

Для добавления узла в граф у него должны быть поля _x, y, id_. Добавим их со значениями координат остановки и osmid

In [135]:
G = nx.MultiDiGraph()       #создаем пустой граф
if "crs" not in G.graph:
    G.graph["crs"] = "EPSG:4326"    #задаем параметр для работы с координатами

for idx, row in nodes.iterrows():
    if not row['bus']=='yes': continue
    stop_id = idx[1]
    row["x"] = row["geometry"].x
    row["y"] = row["geometry"].y
    G.add_node(stop_id, **row.to_dict())

In [169]:
print(G)

MultiDiGraph with 5319 nodes and 0 edges


Таким образом мы смогли получить граф, в который уже добавили все необходимые узлы

### Получение узлов с помощью Overpass-API

В данном разделе мы будем получать данные об узлах через прямое обращение к Overpass-API. Такой подход позволяет проще управлять содержимым узлов, а также обладает большей гибкостью с точки зрения составления запроса

In [2]:
pd.set_option('display.max_columns', None)   #для отображения всех столбцов датафрейма 

nodes_overpass_query = """
[out:json];
area[name="Санкт-Петербург"]->.searchArea;
node["public_transport"="stop_position"](area.searchArea)->.nodes;
(
  .nodes;
);
out body;
>;
out skel qt;
"""
nodes_response = ox._overpass._overpass_request(data={"data": nodes_overpass_query})

Получили все узлы, которым с флагом _stop_position_ (точка на дороге, где останавливается транспорт). Теперь переведём полученный json (уже представляет собой словарь) в датафрейм и добавим столбцы, необходимые для работы с GeoDataFrame (позднее граф будет переведен в этот тип)

In [3]:
nodes_df = pd.DataFrame.from_dict(nodes_response['elements'])

nodes_list = []
for row in nodes_response['elements']:
    tmp_dict = row['tags']
    tmp_dict['osmid']=row['id']
    tmp_dict['x'] = row['lat']
    tmp_dict['y'] = row['lon']
    nodes_list.append(tmp_dict)


nodes_df = pd.DataFrame.from_dict(nodes_list)

Ранее мы получили весь общественный транспорт. Теперь добавим в граф только соответствующие автобусам

In [4]:
G = nx.MultiDiGraph()       #создаем пустой граф
if "crs" not in G.graph:
    G.graph["crs"] = "EPSG:4326"    #задаем параметр для работы с координатами

for idx, row in nodes_df.iterrows():
    if row['bus'] != 'yes': continue
    G.add_node(row['osmid'], **row.to_dict())

In [6]:
print(G.nodes)   #список всех osmid узлов в графе

[223641, 230253, 238796, 244700, 244749, 258978, 259179, 276811, 307668, 327136, 332238, 332441, 332456, 332579, 5508397, 16812601, 16817526, 16820388, 16820408, 21626934, 21752240, 21775300, 21775302, 21775349, 21775505, 21775694, 25264223, 25507282, 25716307, 25720256, 27024506, 27483550, 60081017, 88030395, 94078807, 158691579, 226964222, 233070675, 244503888, 244550320, 245967626, 248812172, 251468419, 251868833, 252654572, 252913816, 259410140, 261652275, 270470475, 272368728, 279161001, 279161009, 302491799, 309354311, 313064963, 313417062, 358119812, 365417713, 378031962, 388561680, 389431514, 389451468, 445713330, 460382153, 502414088, 526967725, 604938960, 652041983, 660851541, 661113360, 698846329, 700371600, 743282581, 765365302, 773602616, 787089064, 787677973, 885309091, 892277402, 914208594, 935195178, 956474399, 956474547, 956474966, 970633186, 975030273, 981542153, 998543245, 1005715597, 1011737010, 1016706733, 1027230289, 1052292775, 1099878891, 1121391838, 1134332800,

### Получение рёбер

Для получения рёбер мы воспользуемся прямым запросом к Overpass-API, с которым "под капотом" работает osmnx. Вероятно, удобнее будет использовать поиск не по области (area), а по полигону, однако это требует проверки

In [7]:
overpass_query = """
    [out:json];
    area[name="Санкт-Петербург"]->.searchArea;
    relation["route"="bus"](area.searchArea)->.routes;
    (
      .routes;
    );
    out body;
    >;
    out skel qt;
    """
routes = ox._overpass._overpass_request(data={"data": overpass_query})

С помощь этого запроса мы получаем словарь с описанием всех отношений, входящих в данную область. Отметим следующие поля:
- _id_ : id отношения
- _members_ : содержит список участников отношения (подробнее в следующей клетке)
- _tags_ : содержит информацию об отношении (название, тип связей и прочую информацию)

Дополнительно разберем поле members, содержащее список участников отношения:
- _node_ : содержит информацию об узлах (_ref_ содержит osmid узла, _role_ содержит роль узла (например, обычная остановка или конечная)
- _way_ : содержит osmid линий на карте OSM, из которых состоит маршрут транспорта

Преобразуем этот словарь в датафрейм:

In [8]:
relations = pd.DataFrame(columns=['relation_id', 'relation_members', 'relation_ways'])

for element in routes["elements"]:
    if element['type'] == 'relation':
        node_list = []
        way_list = []
        
        for member in element['members']:
            if (member['type'] == 'node'): node_list.append(member['ref'])
            elif (member['type'] == 'way'): way_list.append(member['ref'])
                
        relations.loc[len(relations)] = [element['id'], node_list, way_list]

Отметим две вещи важные вещи:
- Отношения однонаправленные. Один маршрут представляется двумя отношениями в разные стороны
- Остановки указаны в порядке следования маршрута (требует проверки)

### Добавление рёбер между вершинами

**_Преполагается, что использовался второй вариант получения списка вершин. Если использовался первый способ, то нужно будет внести изменения в следующий код, однако принцип не изменится_**

Осталось только добавить рёбра в граф. Однако по некоторой причине в списке рёбер есть вершины, которые не встречались в списке вершин. Их добавление недопустимо, поскольку они не содержат необходимых полей. Поэтому добавляем только ребра между уже добавленными вершинами (если какой-то вершины, которую связывает ребро, не окажется в графе, то она будет добавлена автоматически и будет представлять собой только _id_, что недопустимо)

In [9]:
graph_nodes = G.nodes


for idx, members in relations.iterrows():

    members = members['relation_members']
    
    l = -1
    for i in range(len(members)):
        if members[i] in graph_nodes:
            l = i
            break
    if l == -1: 
        continue
    
    r = l + 1
    while r < len(members):
        if members[r] in graph_nodes:
            G.add_edge(members[l], members[r])
            l = r
        r += 1
print(G)

MultiDiGraph with 5319 nodes and 21040 edges


Поскольку далее в коде проекта происходит преобразование графа в типы GeoDataFrame, необходимо проверить возможность преобразования с помощью следующего кода. Если он выполняется, то 

In [10]:
gdf_nodes, gdf_relationships = ox.graph_to_gdfs(G)
print(gdf_relationships)

                                                                      geometry
u           v           key                                                   
223641      1504561628  0    LINESTRING (59.94408 30.30550, 59.94940 30.30302)
                        1    LINESTRING (59.94408 30.30550, 59.94940 30.30302)
230253      365417713   0    LINESTRING (59.95945 30.39443, 59.95944 30.38329)
                        1    LINESTRING (59.95945 30.39443, 59.95944 30.38329)
238796      1728464370  0    LINESTRING (60.19337 29.70333, 60.19129 29.71416)
...                                                                        ...
12284840324 8264796524  0    LINESTRING (59.93551 30.37944, 59.93606 30.37561)
12309692076 9895727770  0    LINESTRING (59.92266 30.29538, 59.92374 30.29907)
12309800079 8069558920  0    LINESTRING (59.83236 30.35834, 59.83242 30.35006)
12309800080 10793763772 0    LINESTRING (59.83290 30.34693, 59.82781 30.34980)
12309800082 3062483735  0    LINESTRING (59.83203 30